# Recurrent Neural Network
После простых методов решения я решил попробовать готовое решение для задачи truecasing-а.

В данном ноутбуке я подготовил train.txt для обучения character-level RNN (а точнее LSTM). <br/>
[Github](https://github.com/raymondhs/pytorch-char-rnn-truecase)<br/>
[Original paper](https://www.aclweb.org/anthology/D16-1225/)<br/>
Изначально я пытался обучить на полных названиях компаний из train.txt, но на 11 мегайбайтах от изначального датасета моя нейронка обучалась 2 часа. Также, когда я применил обученную нейронку на test.txt, мне показало, что 56 мегабайт будут преобразовываться 8 часов, что не особо круто.

In [ ]:
train_companies = open('../kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

In [ ]:
del train_companies[471970]

In [ ]:
import re

def tokenize(company, spaces=False):
    with_spaces = list(filter(None, re.split('(\w+| )', company)))
    if(spaces):
        return with_spaces
    return [token for token in with_spaces if token != ' ']

In [ ]:
with open('input.txt', 'w') as input_file:
    for company in train_companies:
        input_file.write(company + '\n')

## Слова со смешанным регистром
Следом мне пришла другая идея - попробовать обучить нейросеть на словах со смешанным регистром, так как с другими типами слов всё намного проще, однако у данной нейросети не получилось выявить закономерность, лосс не уменьшался и большинство преобразованных слов были просто с заглавной буквы.

In [ ]:
# Поиск слов со смешанным регистром для трейн сета

mixed_words_train = []
for company in train_companies[:-1000000]:
    for word in tokenize(company):
        if not word[1:].islower() and not word[1:].isupper() and len(word) > 1:
            mixed_words.append(word)

In [ ]:
# train set for RNN

with open('input.txt', 'w') as input_file:
    for mixed_word in mixed_words_train:
        input_file.write(mixed_word + '\n')

In [ ]:
# Поиск слов со смешанным регистром для тест сета

mixed_words_test = []
for company in train_companies[-1000000:]:
    for word in tokenize(company):
        if not word[1:].islower() and not word[1:].isupper() and len(word) > 1:
            mixed_words_test.append(word)

In [ ]:
# Test set for RNN

test_lower = open('test.lower.txt', 'w')
test = open('test.txt', 'w')
for mixed_word in mixed_words_test:
    test_lower.write(mixed_word.lower() + '\n')
    test.write(mixed_word + '\n')

## Использование
(Взято из оригинальной документации)

### Training

```
python train.py \
  -data_dir data/wiki \
  -rnn_size 700 \
  -num_layers 3 \
  -dropout 0.25 \
  -batch_size 100 \
  -seq_length 50 \
  -max_epochs 30 \
  -learning_rate 0.001 \
  -checkpoint_dir cv/wiki_lstm_700hidden_3layer \
  -gpuid 0
```

### Truecasing

```
# retrieve best checkpoint on valid data
model=`ls cv/wiki_lstm_700hidden_3layer/*.pt | python best_model.py`

cat data/wiki/test.lower.txt \
| python truecase.py \
    $model \
    -beamsize 10 \
    -verbose 0 \
    -gpuid 0 \
> data/wiki/output.txt

# calculate performance on test set
python word_eval.py data/wiki/test.txt cv/wiki_lstm_700hidden_3layer/output.txt
```